In [1]:
import json

In [2]:
dataset = []
for line in open("Sarcasm_Headlines_Dataset.json", "r"):
    dataset.append(json.loads(line))
    json.load

In [3]:
dataset[0]

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
 'is_sarcastic': 0}

In [4]:
article_link = []
headline = []
is_sarcastic = []

In [5]:
for item in dataset:
    article_link.append(item['article_link'])
    headline.append(item['headline'])
    is_sarcastic.append(item['is_sarcastic'])

In [6]:
is_sarcastic[120]

1

In [7]:
data_len = len(headline)
data_len

26709

In [23]:
import tensorflow as tf
from tensorflow import keras as k
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences as pd
import numpy as np

In [9]:
token = Tokenizer(oov_token="<OOV>")

In [10]:
token.fit_on_texts(headline)

In [11]:
word_index = token.word_index
len(word_index)

29657

In [12]:
seq = token.texts_to_sequences(headline)
padded = pd(seq, padding='post')
padded[0]

array([  308, 15115,   679,  3337,  2298,    48,   382,  2576, 15116,
           6,  2577,  8434,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0])

In [13]:
headline[0]

"former versace store clerk sues over secret 'black code' for minority shoppers"

In [14]:
print(padded.shape)

(26709, 40)


In [38]:
# Train size is 80% of the dataset
# Test size is 20% of the dataset
train_size = round((data_len * 80) / 100)
train_size

21367

In [16]:
train_headline = headline[0:train_size]
test_headline = headline[train_size:]

train_result = is_sarcastic[0:train_size]
test_result = is_sarcastic[train_size:]

In [17]:
token2 = Tokenizer(oov_token="<OOV>")

In [18]:
token2.fit_on_texts(train_headline)
word_index_2 = token2.word_index

train_seq = token2.texts_to_sequences(train_headline)
train_pad = pd(train_seq)

In [19]:
test_seq = token2.texts_to_sequences(test_headline)
test_pad = pd(test_seq)

In [29]:
vocab_size = len(word_index_2) + 1
model = k.Sequential([
    k.layers.Embedding(vocab_size, 50),
    k.layers.GlobalAveragePooling1D(),
    k.layers.Dense(24, activation='relu'),
    k.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
train_pad = np.array(train_pad)
train_result = np.array(train_result)
test_pad = np.array(test_pad)
test_result = np.array(test_result)

In [44]:
training = model.fit(train_pad, train_result, epochs=30, validation_data=(test_pad, test_result), verbose=2)

Train on 21367 samples, validate on 5342 samples
Epoch 1/30
21367/21367 - 9s - loss: 0.0048 - accuracy: 0.9984 - val_loss: 2.1797 - val_accuracy: 0.8149
Epoch 2/30
21367/21367 - 9s - loss: 0.0018 - accuracy: 0.9993 - val_loss: 2.2693 - val_accuracy: 0.8081
Epoch 3/30
21367/21367 - 9s - loss: 5.0737e-05 - accuracy: 1.0000 - val_loss: 2.2646 - val_accuracy: 0.8107
Epoch 4/30
21367/21367 - 9s - loss: 4.5053e-06 - accuracy: 1.0000 - val_loss: 2.2689 - val_accuracy: 0.8104
Epoch 5/30
21367/21367 - 9s - loss: 2.8008e-06 - accuracy: 1.0000 - val_loss: 2.2710 - val_accuracy: 0.8102
Epoch 6/30
21367/21367 - 9s - loss: 2.2523e-06 - accuracy: 1.0000 - val_loss: 2.2751 - val_accuracy: 0.8104
Epoch 7/30
21367/21367 - 9s - loss: 1.9258e-06 - accuracy: 1.0000 - val_loss: 2.2786 - val_accuracy: 0.8104
Epoch 8/30
21367/21367 - 9s - loss: 1.6254e-06 - accuracy: 1.0000 - val_loss: 2.2834 - val_accuracy: 0.8106
Epoch 9/30
21367/21367 - 10s - loss: 1.3831e-06 - accuracy: 1.0000 - val_loss: 2.2878 - val_acc

In [45]:
sentences = [
    'Meh, Kind of good',
    'Climate is perfect'
]

In [49]:
sequences = token2.texts_to_sequences(sentences)
latest_padded = pd(sequences)
model.predict(latest_padded)

array([[1.],
       [0.]], dtype=float32)

In [50]:
 model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 50)          1325400   
_________________________________________________________________
global_average_pooling1d_3 ( (None, 50)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 24)                1224      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 25        
Total params: 1,326,649
Trainable params: 1,326,649
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.evaluate(test_pad, test_result)

5342/5342 [==============================] - 0s 82us/sample - loss: 2.7159 - accuracy: 0.8100


[2.715869650533234, 0.80999625]